## JSON 파일 관리를 위해서 excel 변환
* 해당 JSON은 MIMIC-CXR을 GPT4V를 통해서 Annotation를 만들어서 VQA 형태의 JSON 파일을 만든 데이터셋으로 아래 링크 참고.
https://physionet.org/content/llava-rad-mimic-cxr-annotation/1.0.0/


In [2]:
!pip install openpyxl

In [20]:
import json
import pandas as pd
import os

# JSON 파일 로드
with open("C:/DEV/01. workspace/LLaVA-Med-RAG/data/mimic_cxr_annotaion_json/chat_test_MIMIC_CXR_all_gpt4extract_rulebased_v1.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# 데이터프레임을 위한 리스트
records = []

for entry in data:
    image_download_path = entry.get("image").replace("mimic", "files")
    image_path = entry.get("image")
    image_url = f"https://physionet.org/files/mimic-cxr-jpg/2.0.0/{image_download_path}"
    image_script = f"wget --no-parent --no-directories -N -c --user jeayoung --password 'okok!!8462' -P ./{os.path.dirname(image_path)} \"{image_url}\""
    
    # Report URL 생성
    report_download_folder = os.path.dirname(image_download_path)
    report_folder = os.path.dirname(image_path)
    report_url = f"https://physionet.org/files/mimic-cxr/2.1.0/{report_download_folder}.txt"
    report_script = f"wget --no-parent --no-directories -N -c --user jeayoung --password 'okok!!8462' -P ./{report_folder} \"{report_url}\""
    
    record = {
        "id": entry.get("id"),
        "image_script": image_script,
        "report_script": report_script,
        "image": entry.get("image"),
        "generate_method": entry.get("generate_method"),
        "reason": entry.get("reason"),
        "impression": entry.get("impression"),
        "indication": entry.get("indication"),
        "history": entry.get("history"),
        "view": entry.get("view"),
        "orientation": entry.get("orientation"),
    }
    
    # Chexpert Labels 추가
    chexpert_labels = entry.get("chexpert_labels", {})
    for label, value in chexpert_labels.items():
        record[label] = value
    
    # Conversations에서 human 및 gpt 값 추출 (줄바꿈 유지)
    human_value = next((conv["value"].encode("unicode_escape").decode("utf-8") for conv in entry.get("conversations", []) if conv["from"] == "human"), None)
    gpt_value = next((conv["value"].encode("unicode_escape").decode("utf-8") for conv in entry.get("conversations", []) if conv["from"] == "gpt"), None)
    
    record["human_value"] = human_value
    record["gpt_value"] = gpt_value
    
    # JSON 전체 문자열 (엔터 제거)
    record["json_string"] = json.dumps(entry, separators=(",", ":"))
    
    records.append(record)

# Pandas 데이터프레임 변환
df = pd.DataFrame(records)

# 엑셀로 저장
df.to_excel("C:/DEV/01. workspace/LLaVA-Med-RAG/data/mimic_cxr_annotaion_json/chat_test_MIMIC_CXR_all_gpt4extract_rulebased_v1.xlsx", index=False, engine="openpyxl")

print("엑셀 파일 변환 완료: output.xlsx")


엑셀 파일 변환 완료: output.xlsx


In [21]:
import json
import pandas as pd
import os

# JSON 파일 로드
with open("C:/DEV/01. workspace/LLaVA-Med-RAG/data/mimic_cxr_annotaion_json/chat_train_MIMIC_CXR_all_gpt4extract_rulebased_v1.json", "r", encoding="utf-8") as file:
    data = json.load(file)


# 데이터프레임을 위한 리스트
records = []

for entry in data:
    image_download_path = entry.get("image").replace("mimic", "files")
    image_path = entry.get("image")
    image_url = f"https://physionet.org/files/mimic-cxr-jpg/2.0.0/{image_download_path}"
    image_script = f"wget --no-parent --no-directories -N -c --user jeayoung --password 'okok!!8462' -P ./{os.path.dirname(image_path)} \"{image_url}\""
    
    # Report URL 생성
    report_download_folder = os.path.dirname(image_download_path)
    report_folder = os.path.dirname(image_path)
    report_url = f"https://physionet.org/files/mimic-cxr/2.1.0/{report_download_folder}.txt"
    report_script = f"wget --no-parent --no-directories -N -c --user jeayoung --password 'okok!!8462' -P ./{report_folder} \"{report_url}\""
    
    record = {
        "id": entry.get("id"),
        "image_script": image_script,
        "report_script": report_script,
        "image": entry.get("image"),
        "generate_method": entry.get("generate_method"),
        "reason": entry.get("reason"),
        "impression": entry.get("impression"),
        "indication": entry.get("indication"),
        "history": entry.get("history"),
        "view": entry.get("view"),
        "orientation": entry.get("orientation"),
    }
    
    # Chexpert Labels 추가
    chexpert_labels = entry.get("chexpert_labels", {})
    for label, value in chexpert_labels.items():
        record[label] = value
    
    # Conversations에서 human 및 gpt 값 추출 (줄바꿈 유지)
    human_value = next((conv["value"].encode("unicode_escape").decode("utf-8") for conv in entry.get("conversations", []) if conv["from"] == "human"), None)
    gpt_value = next((conv["value"].encode("unicode_escape").decode("utf-8") for conv in entry.get("conversations", []) if conv["from"] == "gpt"), None)
    
    record["human_value"] = human_value
    record["gpt_value"] = gpt_value
    
    # JSON 전체 문자열 (엔터 제거)
    record["json_string"] = json.dumps(entry, separators=(",", ":"))
    
    records.append(record)

# Pandas 데이터프레임 변환
df = pd.DataFrame(records)

# 파일 분할 저장 (10개로 나누기)
num_files = 10
chunk_size = len(df) // num_files + (len(df) % num_files > 0)


for i in range(num_files):
    chunk_df = df[i * chunk_size:(i + 1) * chunk_size]
    file_path = os.path.join(f"C:/DEV/01. workspace/LLaVA-Med-RAG/data/mimic_cxr_annotaion_json/chat_train_MIMIC_CXR_all_gpt4extract_rulebased_v1_{i+1}.xlsx")
    chunk_df.to_excel(file_path, index=False, engine="openpyxl")
    print(f"엑셀 파일 저장 완료: {file_path}")


엑셀 파일 저장 완료: C:/DEV/01. workspace/LLaVA-Med-RAG/data/mimic_cxr_annotaion_json/chat_train_MIMIC_CXR_all_gpt4extract_rulebased_v1_1.xlsx
엑셀 파일 저장 완료: C:/DEV/01. workspace/LLaVA-Med-RAG/data/mimic_cxr_annotaion_json/chat_train_MIMIC_CXR_all_gpt4extract_rulebased_v1_2.xlsx
엑셀 파일 저장 완료: C:/DEV/01. workspace/LLaVA-Med-RAG/data/mimic_cxr_annotaion_json/chat_train_MIMIC_CXR_all_gpt4extract_rulebased_v1_3.xlsx
엑셀 파일 저장 완료: C:/DEV/01. workspace/LLaVA-Med-RAG/data/mimic_cxr_annotaion_json/chat_train_MIMIC_CXR_all_gpt4extract_rulebased_v1_4.xlsx
엑셀 파일 저장 완료: C:/DEV/01. workspace/LLaVA-Med-RAG/data/mimic_cxr_annotaion_json/chat_train_MIMIC_CXR_all_gpt4extract_rulebased_v1_5.xlsx
엑셀 파일 저장 완료: C:/DEV/01. workspace/LLaVA-Med-RAG/data/mimic_cxr_annotaion_json/chat_train_MIMIC_CXR_all_gpt4extract_rulebased_v1_6.xlsx
엑셀 파일 저장 완료: C:/DEV/01. workspace/LLaVA-Med-RAG/data/mimic_cxr_annotaion_json/chat_train_MIMIC_CXR_all_gpt4extract_rulebased_v1_7.xlsx
엑셀 파일 저장 완료: C:/DEV/01. workspace/LLaVA-Med-RAG/data/mi